In [1]:
import numpy as np
import pandas as pd
import os
import multiprocessing as mul
from multiprocessing import Process
import matplotlib.pyplot as plt
#import seaborn as sns
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from IPython.display import clear_output
from scipy.stats import chi2
from scipy.stats import norm
from scipy.stats import gaussian_kde
import scipy.stats as sct
from astropy.coordinates import SkyCoord as scr
from astropy import units as u
from time import sleep
from numba import jit

In [2]:
####
#### IMPORTING AND SPLITTING ICDATA $$$


path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/events"
t_eff_path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/uptime"
irf_path = "/media/darkwake/VIB2/Project-IceCube/icecube_10year_ps/irfs"
filenames = ["IC40_exp.csv", "IC59_exp.csv","IC79_exp.csv", "IC86_I_exp.csv", "IC86_II_exp.csv",
"IC86_III_exp.csv", "IC86_IV_exp.csv", "IC86_V_exp.csv", "IC86_VI_exp.csv", "IC86_VII_exp.csv"]
file = filenames[0]
f = open(os.path.join(path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
content = []
for file in filenames:
    f = open(os.path.join(path, file), 'r')
    lines = f.readlines()
    #print(len(lines) - 1)
    for line in lines[1:]:
        content.append(line.split())
    f.close()
icdata = pd.DataFrame(content, columns=column, dtype=float)#.convert_dtypes(infer_objects=True,convert_integer=True,convert_floating=True)
icdata['log10(E/GeV)'] = [float(i) for i in icdata['log10(E/GeV)']]
icdata['MJD[days]'] = [float(i) for i in icdata['MJD[days]']]

print("read icdata")
f.close()




read icdata



#Importing UPtime data
file = filenames[0]
f = open(os.path.join(t_eff_path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
uptdata = []
for file in filenames:
    content = []
    f = open(os.path.join(t_eff_path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
    temp = pd.DataFrame(content, columns=column)
    temp['MJD_start[days]'] = [float(i) for i in temp['MJD_start[days]']]
    temp['MJD_stop[days]'] = [float(i) for i in temp['MJD_stop[days]']]
    uptdata.append(temp)
    temp = []
    content = []
    print(file + " read")
f.close()


#Importing Aeff data
filenames = ["IC40_effectiveArea.csv", "IC59_effectiveArea.csv","IC79_effectiveArea.csv", "IC86_I_effectiveArea.csv", "IC86_II_effectiveArea.csv"]
file = filenames[0]
f = open(os.path.join(irf_path, file), 'r')
lines = f.readlines()
column=lines[0].split()
column.pop(0)
eadata = []
for file in filenames:
    content = []
    f = open(os.path.join(irf_path, file), 'r')
    lines = f.readlines()
    for line in lines[1:]:
        content.append(line.split())
    f.close()
    temp = pd.DataFrame(content, columns=column, dtype=float)
    #temp['MJD_start[days]'] = [float(i) for i in temp['MJD_start[days]']]
    #temp['MJD_stop[days]'] = [float(i) for i in temp['MJD_stop[days]']]
    eadata.append(temp)
    temp = []
    content = []
    print(file + " read")
f.close()

icdata.dtypes

eadata[0].dtypes

icwidths = [int(i) for i in "0 36900 107011 93133 136244 112858 122541 127045 129311 123657 145750".split(' ')]
print(np.sum(icwidths))

In [3]:
icdata

,MJD[days],log10(E/GeV),AngErr[deg],RA[deg],Dec[deg],Azimuth[deg],Zenith[deg]
0,54562.379113,3.31,0.48,203.024,16.760,218.374,106.712
1,54562.386622,5.17,0.62,75.024,-13.371,349.100,76.646
2,54562.387892,3.92,0.56,97.501,11.290,327.060,101.291
3,54562.392677,5.57,1.13,305.857,-33.932,120.413,56.090
4,54562.404776,5.80,1.11,143.516,-38.279,287.171,51.684
...,...,...,...,...,...,...,...
1134445,58307.961110,5.14,0.20,221.246,-59.959,141.239,29.956
1134446,58307.962666,2.94,1.23,191.547,4.415,171.627,94.312
1134447,58307.963556,2.88,0.30,308.127,42.543,55.437,132.606
1134448,58307.963972,2.98,0.21,332.199,44.720,31.489,134.811


ictimes = [float(i) for i in icdata['MJD[days]']]
icparts = [np.sum(icwidths[:i]) for i in range(1,len(icwidths)+1)]
#icparts.pop(-1)
icparts

upt_icparts = icparts[:5]
upt_icparts.append(icparts[-1])
len(upt_icparts) - 1 == len(eadata)

log_e = np.round(np.arange(2, 10.2, 0.2), 2)
dec_nu = list(set(eadata[0]['Dec_nu_min[deg]'].values).union(set(eadata[0]['Dec_nu_max[deg]'].values)))
dec_nu.sort()
dec_nu = np.array(dec_nu)

nu_e = 5
e_ind = 0
for i in range(len(log_e) - 1):
    if nu_e >= log_e[i] and nu_e < log_e[i+1]:
        e_ind = i
print([e_ind, i])

In [4]:
#IMPORTING MSPDATA
f = open("/media/darkwake/VIB2/Project-IceCube/allpsr1.68.txt", 'r')
lines = f.readlines()
content=[]
column=lines.pop(0).replace('x', '').replace('#', '').split()
for line in lines[:]:
    content.append(line.split())
    #the INITAL DATABASE IS CLUTTERED SO WE REMOVE THE NULL COLUMNS AND OTHER CLUTTER
f.close()
mspdata = pd.DataFrame(content).drop(0, axis=1)#.dropna()#.drop_duplicates()#.drop(range(0,6)).dropna()

line = []
lines = []
mspdata.columns = column
column = []
content=[]
#mspdata = mspdata.sort_values('DECJD')
mspdata.dropna(inplace=True)
#mspdata = mspdata.reset_index()
#mspdata = mspdata.drop("index", axis=1)

In [5]:
mspdata

,NAME,RAJD,DECJD
0,J0002+6216,0.74238,62.26928
1,J0006+1834,1.52,18.5831
2,J0007+7303,1.7571,73.0521
3,J0011+08,2.9,8.17
4,B0011+47,3.57396,47.77594
...,...,...,...
3336,J2351+8533,357.764,85.55573
3337,J2352+65,358,65
3338,J2354-22,358.60833,-22.86472
3339,B2351+61,358.5199292,61.9296792


In [6]:
msra = np.array([float(i) for i in mspdata['RAJD'].values])
msdec = np.array([float(i) for i in mspdata['DECJD'].values])
icra = np.array([float(i) for i in icdata['RA[deg]']])
icdec = np.array([float(i) for i in icdata['Dec[deg]']])
icang = np.array([float(i) for i in icdata['AngErr[deg]']])
global p, lg
p = len(msra)
lg = len(icra) // p + 1
#cut = 20
#cone = 3
#psrno = 0

In [7]:
[min(msdec), max(msdec)]

[-83.298894, 86.720335]

In [8]:
count = 0
for i in range(p):
    if abs(msdec[i] - 90) <= 5:
        count+=1

count

2

In [9]:
icra[[0, 6, 8] ]

array([203.024, 147.017, 236.214])

In [10]:
@jit(nopython=True)
def hvovec(lon1=0.0, lat1=0.0, lon2=0.0, lat2=0.0, rad=False):

    '''
    Parameters
    ----------
    lon1 : float
        Longitude of first point.
    lat1 : float
        Latitude of first point.


    lon2 : float
        Longitude of second point.
    lat2 : float
        Latitude of second point.

    rad : Boolean, optional
        (default) False -> Angle is returned in Degrees
        True -> Angle is returned in radians
    

    Returns
    -------
    float
        Returns the haversine angle between two vectors given their latitude and longitude
    

    Warning
    -------
        This function assumes the input to be in degrees and of equal length\n
        or one of the input pairs to be a single value and the other pair to be an array
    '''

    #Convert decimal degrees to Radians:
    lon1 = np.deg2rad(lon1)
    lat1 = np.deg2rad(lat1)
    lon2 = np.deg2rad(lon2)
    lat2 = np.deg2rad(lat2)

    #Implementing Haversine Formula: 
    dlon = np.subtract(lon2, lon1)
    #dlat = np.subtract(lat2, lat1)

    a = np.add(np.multiply(np.sin(lat1), np.sin(lat2)), np.multiply(np.multiply(np.cos(lat1), np.cos(lat2)), np.cos(dlon)))

    if rad == True:
        return np.arccos(a)
    else:
        return np.abs(np.rad2deg(np.arccos(a)))


## SINGLE SOURCE ANALYSIS

In [11]:
@jit(nopython=True)
def angdecfinder(psrno, declcut):

    '''

    Parameters
    ----------
    psrno : int
        Pulsar number from the ATNF catalogue.
    
    declcut : float
        Declination cut-off in degrees.



    Returns
    -------
    float
    Returns the absolute difference between the declinations of pulsar {psrno}\n
    and each of the neutrinos within the specified declcut
    '''

    #aang = hvovec(msra[psrno], msdec[psrno], icra, icdec)
    decdf = list(np.abs(np.subtract(icdec, icdec[psrno])))
    nuind = []
    for i in range(0, len(decdf)):
        if decdf[i] < declcut:
            nuind.append(i)
            #decdf.pop(i)
            #aang[i] = -1
    #fdecdf = list(np.abs(np.subtract(icdec[nuind], icdec[psrno])))
    #return [nuind, decdf]
    return nuind

@jit(nopython=True)
def S_ij(psrno, nuind_inp): 

    '''
    Parameters
    ----------
    psrno : int
        Pulsar number from the ATNF catalogue.
    
    nuind_inp : list
    Returns the signal PDF for the {psrno}th pulsar 
    and the all neutrinos within the specified declcut as given in {nuind}
    '''
    nuind = list(nuind_inp)
    for i in nuind:
        if i == -1:
            nuind.pop(i)
    ang2 = hvovec(msra[psrno], msdec[psrno], icra[nuind], icdec[nuind], rad=True) ** 2
    sg = np.deg2rad(icang[nuind]) ** 2
    return np.exp(-1 * ang2 / (2 * sg)) / (2 * np.pi * sg)


@jit(nopython=True)
def bgs(psrno, cone, twopi = False):

    '''
    Returns the background PDF for a given pulsar {psrno} within\n
    a cone of {cone} degrees for SINGLE STACKED ANALYSIS
    '''

    if twopi == True:
        s_ang = (np.sin(np.deg2rad(msdec[psrno] + cone)) - np.sin(np.deg2rad(msdec[psrno] - cone)))*(2 * np.pi)
        return 1/s_ang
    else:
        s_ang = (np.sin(np.deg2rad(msdec[psrno] + cone)) - np.sin(np.deg2rad(msdec[psrno] - cone)))
        return 1/s_ang

@jit(nopython=True)
def Pr(x, Ns, S, B):

    '''
    Parameters
    ----------
    x : int
        Assumed no.of associated events

    Ns : int
        No.of neutrinos used for analysis

    S : float
        Signal PDF
    B : float
        Background PDF
    
    Returns
    -------
    float 
        Returns the probability of the selected set of neutrinos being associated\n
        with a given pulsar with {Ns} neutrinos, {S} signal and {B} background PDF and {x} assumed associated events
    '''

    nsN = x/Ns
    return np.add(np.multiply(nsN , S), np.multiply(np.subtract(1, nsN), B))

@jit(nopython=True)
def TS(i, x, S, B, Ns):

    '''Returns the Test Stastic value fot {i}^th grb
    at $n_s$ = {x} for its parameters S, B, Ns    '''
    
    return 2*np.sum(np.log(Pr(x,  Ns, S, B)/B))


def ns_for_TSmax(i, S, B, Ns):

    '''
    Returns the value of $n_s$ for which
    the TS is maximum for {i}^th grb
    '''

    #returns the TSmax for i^th GRB
    nll = lambda x: -TS(i,x, S, B, Ns)
    soln = minimize(nll, 6 ,bounds=((0,None),))
    ns = float(soln.x)
    return ns


In [12]:
angdecfinder(0, 5)

[0,
 9,
 10,
 13,
 22,
 33,
 42,
 46,
 53,
 57,
 59,
 62,
 64,
 66,
 69,
 82,
 119,
 132,
 157,
 159,
 168,
 196,
 209,
 249,
 256,
 264,
 288,
 293,
 295,
 302,
 328,
 329,
 331,
 336,
 341,
 383,
 399,
 406,
 429,
 430,
 437,
 450,
 465,
 477,
 482,
 502,
 508,
 524,
 556,
 569,
 592,
 594,
 613,
 633,
 636,
 637,
 654,
 699,
 708,
 723,
 736,
 788,
 821,
 835,
 845,
 859,
 874,
 878,
 901,
 913,
 915,
 926,
 946,
 947,
 983,
 987,
 992,
 1001,
 1017,
 1027,
 1034,
 1036,
 1049,
 1059,
 1060,
 1068,
 1085,
 1096,
 1097,
 1108,
 1129,
 1149,
 1166,
 1175,
 1183,
 1194,
 1198,
 1237,
 1250,
 1259,
 1265,
 1277,
 1283,
 1299,
 1317,
 1323,
 1363,
 1376,
 1394,
 1413,
 1429,
 1439,
 1448,
 1481,
 1527,
 1575,
 1585,
 1586,
 1617,
 1653,
 1658,
 1670,
 1674,
 1691,
 1692,
 1701,
 1705,
 1708,
 1709,
 1717,
 1736,
 1743,
 1762,
 1767,
 1775,
 1790,
 1808,
 1812,
 1856,
 1857,
 1954,
 1962,
 1977,
 1990,
 2001,
 2007,
 2011,
 2017,
 2036,
 2050,
 2054,
 2071,
 2091,
 2100,
 2109,
 2146,
 21

In [13]:
S_ij(0, angdecfinder(0, 5))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(readonly array(float64, 1d, C), list(int64)<iv=None>)
 
There are 22 candidate implementations:
[1m    - Of which 20 did not match due to:
    Overload of function 'getitem': File: <numerous>: Line N/A.
      With argument(s): '(readonly array(float64, 1d, C), list(int64)<iv=None>)':[0m
[1m     No match.[0m
[1m    - Of which 2 did not match due to:
    Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 166.
      With argument(s): '(readonly array(float64, 1d, C), list(int64)<iv=None>)':[0m
[1m     Rejected as the implementation raised a specific error:
       NumbaTypeError: [1munsupported array index type list(int64)<iv=None> in [list(int64)<iv=None>][0m[0m
  raised from /home/darkwake/.local/lib/python3.10/site-packages/numba/core/typing/arraydecl.py:72


In [18]:
#@jit(nopython=True)
def singpsr(psrno):
    cut = 5
    cone = 5
    nuind= angdecfinder(psrno, cut)
    S = S_ij(psrno, nuind)
    B = bgs(psrno, cone, True)
    Ns = len(nuind)
    maxns = ns_for_TSmax(psrno, S, B, Ns)
    return [maxns, np.sqrt(TS(psrno, maxns, S, B, Ns))]

tsmax = []
errpsr = []
pool = mul.Pool(8, maxtasksperchild=200)
op_async = pool.map_async(singpsr, range(p))
tsmax = op_async.get()
op_async = []
pool = []
'''
for i in range(p):
    #try:
    tsmax.append(singpsr2(i))
    #except TypeError:
     #   errpsr.append(i)'''
maxns = []
sqrtts = []
for i in tsmax:
    maxns.append(i[0])
    sqrtts.append(i[1])



TypingError: [1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(readonly array(float64, 1d, C), list(int64)<iv=None>)
 
There are 22 candidate implementations:
[1m   - Of which 20 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(readonly array(float64, 1d, C), list(int64)<iv=None>)':[0m
[1m    No match.[0m
[1m   - Of which 2 did not match due to:
   Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 166.
     With argument(s): '(readonly array(float64, 1d, C), list(int64)<iv=None>)':[0m
[1m    Rejected as the implementation raised a specific error:
      NumbaTypeError: [1munsupported array index type list(int64)<iv=None> in [list(int64)<iv=None>][0m[0m
  raised from /home/darkwake/.local/lib/python3.10/site-packages/numba/core/typing/arraydecl.py:72
[

sqrtt = np.sqrt(sqrtts)

np.argmax(sqrtt)

psrno = 0
cut = 5
cone = 5
nuind, decdf = angdecfinder(psrno, cut)
S = S_ij(psrno, nuind)
B = bgs(psrno, cone, True)
Ns = len(nuind)
maxns = ns_for_TSmax(psrno, S, B, Ns)

len(S)

singpsr(971)

def plotpsr(psrno):
    cut = 5
    cone = 5
    nuind, decdf = angdecfinder(psrno, cut)
    S = S_ij(psrno, nuind)
    B = bgs(psrno, cone)
    Ns = len(nuind)
    tns=[TS(psrno, i, S, B, Ns) for i in range(0, 1000)]
    print(Ns)
    plt.plot(range(0, 1000), tns)
    plt.show()
    print([ns_for_TSmax(psrno, S, B, Ns), TS(psrno, ns_for_TSmax(psrno, S, B, Ns), S, B, Ns)])
plotpsr(971)

def psrcustom(arr):
    cut = 5
    cone = 5
    for psrno in arr:
        nearcrd=[]
        for i in range(len(icdec)):
            if abs(icdec[i] - icdec[psrno]) <= cut:
                nearcrd.append([icra[i], icdec[i], icang[i]])
        print([psrno, msra[psrno], icdec[psrno]])
        np.savetxt(f"outputs/ndecnear{psrno}.txt", nearcrd)
        print("len(nearcrd) = ", len(nearcrd))

psrcustom([971])

## STACKING ANALYSIS

gamma = 2.0
e_nu = 1e11 ** gamma
weights = [aeff[i] * uptime[i] * e_nu *0.2  for i in range(len(aeff))]

In [34]:
len(dec_nu)

51

![SNOWFALL](test.png)

In [16]:
e_nu = 1e9**(log_e[:-1] + 0.1)
de_nu = 1e9**0.2

In [17]:
len(e_nu)

40

In [25]:
def psr_wt_quick(nusample_wall, psrno, gamma = 1):

    '''
    Returns the weight of psrno^th pulsar for a given neutrino sample {nusample_wall}
    '''
    
    aeff = -1
    #nu_e = icdata['log10(E/GeV)'].values[nu]
    psr_decl = icdec[psrno]
    #upt = upt_icparts[nusample_wall]
    t_upt = uptdata[nusample_wall]['MJD_stop[days]'].values[-1] - uptdata[nusample_wall]['MJD_start[days]'].values[0]
    t_upt*=86400
    d_ind = 0
    for i in range(len(dec_nu) - 1):
        if dec_nu[i] <= psr_decl and dec_nu[i+1] >= psr_decl:
            d_ind = i
            break
    ea_temp = eadata[nusample_wall][d_ind*40:(d_ind+1)*40]
    earea = np.array(ea_temp['A_Eff[cm^2]'].values)
    weight_kj = t_upt * np.sum(np.multiply(np.multiply(earea, np.power(e_nu, gamma)), de_nu))
    sleep(1e-8)
    return weight_kj                           
            

all_weights[i][j] = weight of pulsar i in j^th partition of uptime data (upt_part)

In [42]:
all_weights = []
for i in range(len(upt_icparts)-1):
    print(upt_icparts[i])
    all_weights.append([psr_wt_quick(i, psrno) for psrno in range(p)])

sum_weights = [np.sum(i) for i in all_weights]
sum_weights

0
36900
143911
237044
373288


[2.279424596694289e+109,
 2.773500686026117e+100,
 2.8222856321077403e+100,
 3.671917575973628e+100,
 3.4982131440540488e+100]

In [43]:
def S_ij(nu): 

    '''
    Returns the signal PDF for the {psrno}th pulsar and nuind_inp neutrino
    '''
    ang2 = hvovec(msra, msdec, icra[nu], icdec[nu]) ** 2#, icra[nuind], icdec[nuind], rad=True) ** 2
    sg = np.deg2rad(icang[nu]) ** 2
    return np.exp(-1 * ang2 / (2 * sg)) / (2 * np.pi * sg)


def S_i(nu):

    '''
        Parameters
        ----------
        nu : int
            Index of the neutrino in the sample

        Returns
        -------
        Returns the signal PDF for the {nu}th neutrino with all pulsars 
        i.e S_i = wieghted-sum_j S_ij
    '''



    sij = S_ij(nu)
    wall = 0
    for i in range(len(upt_icparts)-1):
        if upt_icparts[i] <= nu and upt_icparts[i+1] > nu:
            wall = i
            break
    sleep(1e-8)
    return np.dot(sij, all_weights[wall] ) / sum_weights[wall]

for 
all_sij = [S_ij(psrno, nuind) for psrno in range(p)]

![SNOWFALL](bg.png)

### For this project, Cone is $5{\degree}$

In [44]:
def Bi_stacked(nu, cone):

    '''
    Parameters
    ----------
    nu : int
        Index of the neutrino from IceCube sample
    cone : float
        Cone angle in degrees.
    

    Returns
    -------
    float
        Returns the background PDF for the {nu}th neutrino
    '''

    count = 0
    for i in range(len(icdec)):
        if abs(icdec[i] - icdec[nu]) <= cone:
            count+=1
    binwidth = (np.sin(np.deg2rad(icdec[nu] + cone)) - np.sin(np.deg2rad(icdec[nu] - cone)))*2*np.pi
    sleep(1e-8)
    return count / (binwidth*len(icdata))


def TS_st(x, S, B, Ns):

    '''
    Returns
    ----------
        Returns the Test Stastic value at
        $n_s$ = {x} for its parameters S, B, Ns
    '''
    if x >=0:
        return 2*np.sum(np.log(Pr(x,  Ns, S, B)/B))
    else:
        return -1


def ns_for_TSmax_st(S, B, Ns):
    '''
    Returns the value of $n_s$ for which
    the TS is maximum for {i}^th grb
    '''

    #returns the TSmax for i^th GRB
    nll = lambda x: -TS_st(x, S, B, Ns)
    soln = minimize(nll, 3 , bounds = [(0, None)], tol=1e-12)
    ns = np.round(soln.x, 6)[0]
    #print(soln.success)
    return ns

Find 

In [46]:
cut = 5
cone = 5
#nuind, ang = angfinder(psrno, cut)
def sigbag_nu(nu):
    '''
        Returns the signal and background PDF for the {nu}th neutrino
    '''

    sleep(1e-8)
    if nu % 100000 == 0:
        print(nu)
    return [S_i(nu), Bi_stacked(nu, cone)]

pool = mul.Pool(8, maxtasksperchild=100)
op_async = pool.map_async(sigbag_nu, range(len(icdec)//5))
tmp = op_async.get()
pool.close()
pool = []
op_async = []
S_all = tmp[:,0]
B_all = tmp[:,1]


0
100000


Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-15:
Process ForkPoolWorker-11:
Process ForkPoolWorker-13:
Process ForkPoolWorker-12:
Process ForkPoolWorker-16:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    s

This method takes ~30 minutes for 100000 neutrinos

$\implies$ 1134450 neutrinos take ~6.5 to 7hrs!!

UNACCEPTABLE!!!

NEED FASTER METHOD

In [ ]:
#S = [S_i(i) for i in range(len(icdata))]
#B = [Bi_stacked(i, cone) for i in range(len(icdata))]
#nuind = np.array(nuind)
Ns = len(icdata)#np.count_nonzero(nuind+1)
Ts_arr = lambda x: np.sum([TS_st(x, S_all[i], B_all[i], Ns) for i in range(len(icdata))])
nll = lambda x: -Ts_arr(x, S, B, Ns)
soln = minimize(nll, 3 , bounds = [(0, None)], tol=1e-12)
ns = np.round(soln.x, 6)[0]

#tns=[Ts_arr(i) for i in range(0, 1000)]
#print([ns_for_TSmax(psrno, S, B, Ns), TS(psrno, ns_for_TSmax(psrno, S, B, Ns), S, B, Ns), TS(psrno, 0, S, B, Ns)])
#print(Ns)

In [ ]:
B

In [ ]:
def plotpsr_st(psrno):
    cut = 5
    cone = 5
    nuind, ang = angfinder(psrno, cut)
    S = sig_fn(psrno)
    B = bgs(psrno, cone)
    nuind = np.array(nuind)
    Ns = len(icdata)
    tns=[TS(psrno, i, S, B, Ns) for i in range(0, 1000)]
    print(Ns)
    plt.plot(range(0, 1000), tns)
    plt.show()
    print([ns_for_TSmax(psrno, S, B, Ns), TS(psrno, ns_for_TSmax(psrno, S, B, Ns), S, B, Ns), TS(psrno, 0, S, B, Ns)])
plotpsr_st(1)

In [ ]:
weights = np.array(weights)


def singpsr_stacked(psrno):
    #cut = 5
    cone = 5
    S = sig_fn(psrno)
    B = bgs(psrno, cone)
    Ns = len(icdata)#np.count_nonzero(nuind+1)
    maxns = ns_for_TSmax(psrno, S, B, Ns)
    sqrtts = np.sqrt(TS(psrno, np.round(maxns, 4), S, B, Ns))
    return [np.round(maxns, 4), np.round(sqrtts, 4)]

In [ ]:
aa

In [ ]:
tsmax = []
pool = mul.Pool(8, maxtasksperchild=800)
op_async = pool.map_async(singpsr_stacked, range(p))
tsmax = op_async.get()
op_async = []
pool = []
maxns = []
maxts = []
for i in tsmax:
    maxns.append(i[0])
    maxts.append(i[1])

In [ ]:
binnedsts, wsts = np.histogram(maxts, 10)

ne01 = 3
amp  = p/3.0#np.sum(binnedsts)/ne01
#binnedsts = binnedsts/amp
yerr = np.sqrt(binnedsts)
b2 = binnedsts[(binnedsts>10)]
yerr2 = yerr[(binnedsts>10)]
x=0.5*(wsts[1:]+ wsts[:-1])
x2 = x[(binnedsts>10)]
xerr=x[0]-wsts[0]

In [ ]:
yerr

In [ ]:
wsts

In [ ]:
#gaussian = lambda x, mean, var:  amp*norm.pdf(x, mean, var)
def gaussian(x,mean,sd):
#    mean=0.114
    A=p/3.0
    return A*np.exp(-(x - mean) ** 2.0 / sd ** 2.0)
loggaussian = lambda x, mean, var:  np.log(gaussian(x, mean, var))
gparam, gerr = curve_fit(gaussian, x2, b2,  p0=[0.1, 1.0],sigma=yerr2, absolute_sigma=True, maxfev = 10000, method='dogbox')
gx = np.linspace(0, wsts[-1], 100)

In [ ]:
gparam

In [ ]:
max(maxts)

In [ ]:
font = {'family': 'serif',
        'color':  'black',
        'weight': 'bold',
        'size': 22,
        }
plt.figure(figsize=(10.8,7.2))
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], fontsize=17, fontweight='bold')
plt.yscale('log')
plt.yticks([1, 10, 100, 1000, 10000], [1, 10, 100, 1000, 10000], fontsize=17, fontweight='bold')

plt.errorbar(x, binnedsts, xerr=xerr,yerr=yerr,  aa = True, barsabove = True, fmt='ob',ecolor='blue', elinewidth=1.35, label='$\sqrt{TS_{max}}$')
plt.plot(gx,gaussian(gx, gparam[0], gparam[1]),'--', alpha=0.6, label=f'Gaussian Fit', color='darkred', lw=2.2)

plt.xlabel("$\mathbf{\sqrt{TS_{max}}}$", fontweight='bold', fontdict=font)
plt.ylabel("No.of Events", fontweight='bold', fontdict=font)
#plt.grid(True, which='major')
legend =plt.legend(prop={'size':17}, framealpha=0)
legend.get_frame().set_facecolor('none')
plt.savefig("outputs/sqrt(TSmax)stacked.pdf")
plt.show()